In [1]:
!pip install qwen-vl-utils yt-dlp flash-attn --no-build-isolation accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 98.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 57.8 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.7.2.post1-cp310-cp310-linux_x86_64.whl size=190160474 sha256=0b454d9e650bfc437cc71335080172a5d05f51eab355636c9d5b7321fec7318e
  Stored in directory: /root/.cache/pip/wheels/da/ec/5b/b2c37a8e4f755ad82492a822463bca0817f0e0e11de874b550
Successfully built flash-attn


In [14]:
from yt_dlp import YoutubeDL

YOUTUBE_URL = "https://www.youtube.com/watch?v=EI4ks4VdQBA"

ydl_opts = {
    "format": "best",
    "outtmpl":"sample.mp4"
    }
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([YOUTUBE_URL])

[youtube] Extracting URL: https://www.youtube.com/watch?v=EI4ks4VdQBA
[youtube] EI4ks4VdQBA: Downloading webpage
[youtube] EI4ks4VdQBA: Downloading ios player API JSON
[youtube] EI4ks4VdQBA: Downloading mweb player API JSON
[youtube] EI4ks4VdQBA: Downloading m3u8 information
[info] EI4ks4VdQBA: Downloading 1 format(s): 18
[download] Destination: sample.mp4
[download] 100% of    1.18MiB in 00:00:00 at 14.10MiB/s  


In [3]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

MODEL_NAME = "Qwen/Qwen2-VL-7B-Instruct"

model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="flash_attention_2",
)

processor = AutoProcessor.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [15]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "sample.mp4",
                "max_pixels": 1280 * 780,
                "fps": 0.1,
            },
            {"type": "text", "text": "What's happening in the video? who wins the penalty shootout?."},
        ],
    }
]

text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

image_inputs, video_inputs = process_vision_info(messages)

inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

generated_ids = model.generate(**inputs, max_new_tokens=512)

generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]

output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

print(output_text)

['The video shows a penalty shootout between Argentina and France. Argentina wins the shootout and celebrates with their fans.']


In [9]:
!pip install gradio


In [17]:
import gradio as gr

def process_video_query(query):

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "video",
                    "video": "sample.mp4",
                    "max_pixels": 1280 * 780,
                    "fps": 0.1,
                },
                {"type": "text", "text": query},
            ],
        }
    ]

    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    generated_ids = model.generate(**inputs, max_new_tokens=512)
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in
        zip(inputs.input_ids, generated_ids)
    ]

    output_text = processor.batch_decode(
        generated_ids_trimmed,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )

    return output_text[0]

with gr.Blocks() as demo:
    gr.Markdown("# Chat with Video")

    with gr.Row():

        gr.Video("sample.mp4", label="Current Video")

    with gr.Row():
        text_input = gr.Textbox(
            label="Ask a question",
            placeholder="E.g., What's happening in this video?",
            lines=2
        )

    with gr.Row():
        submit_btn = gr.Button("Ask")
        clear_btn = gr.ClearButton()

    output = gr.Textbox(label="Response", lines=4)


    submit_btn.click(
        fn=process_video_query,
        inputs=text_input,
        outputs=output
    )

if __name__ == "__main__":
    demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dc66e599f94aebe540.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
